# Indmat prosjekt
###### Liva Berge Flo, André Pettersen-Dahl, Herman Neple

#### Oppgave 1

#### Oppgave 3

Vi begynner med å importere alle bibliotek vi trenger

In [1]:
from layers import *
from neural_network import NeuralNetwork
from utils import onehot
import numpy as np
from data_generators import get_train_test_sorting

In [2]:
#definerer variabler
r = 5
m = 2

d = 10
k = 5
p = 15
L = 2

embed = EmbedPosition(9,m,d)
att1 = Attention(d,k)
ff1 = FeedForward(d,p)
un_embed = LinearLayer(d,m)
softmax = Softmax()
loss = CrossEntropy()

nn = NeuralNetwork([embed, att1, ff1, un_embed, softmax])

data = get_train_test_sorting(r, m, samples_per_batch=250,n_batches_train=10, n_batches_test=4)

x_train = data['x_train']
y_train = data['y_train']

for j in range(50):
    losses = []
    for i in range(10):
        X = onehot(x_train[i], m)
        Z = nn.forward(X)

        losses.append(loss.forward(Z,y_train[i]))
        dLdZ = loss.backward()
        nn.backward(dLdZ)
        nn.step_Adam()
    mean_loss = np.mean(losses)
    print(f'Iterasjon {j} L = {mean_loss}')

print()

Iterasjon 0 L = 0.6764427103811563
Iterasjon 1 L = 0.5825923585711106
Iterasjon 2 L = 0.4819932305338076
Iterasjon 3 L = 0.44701041154600507
Iterasjon 4 L = 0.4509062844954621
Iterasjon 5 L = 0.42945274709859227
Iterasjon 6 L = 0.4152755966693008
Iterasjon 7 L = 0.42335742910537916
Iterasjon 8 L = 0.4138735340400127
Iterasjon 9 L = 0.40409670403037146
Iterasjon 10 L = 0.38713951171093114
Iterasjon 11 L = 0.3747283439135328
Iterasjon 12 L = 0.3654478585527182
Iterasjon 13 L = 0.37703705204019516
Iterasjon 14 L = 0.3656813260500883
Iterasjon 15 L = 0.35880116655015154
Iterasjon 16 L = 0.36144752565203764
Iterasjon 17 L = 0.35958237944417915
Iterasjon 18 L = 0.3350525634741123
Iterasjon 19 L = 0.34021374761207185
Iterasjon 20 L = 0.3332916156095082
Iterasjon 21 L = 0.3107956480733405
Iterasjon 22 L = 0.3101593519439032
Iterasjon 23 L = 0.31796954449369064
Iterasjon 24 L = 0.38054754261735485
Iterasjon 25 L = 0.3836759580587365
Iterasjon 26 L = 0.37112445748786305
Iterasjon 27 L = 0.352463

In [3]:
for j in range(50):
    losses = []
    for i in range(10):
        X = onehot(x_train[i], m)
        Z = nn.forward(X)

        losses.append(loss.forward(Z,y_train[i]))
        dLdZ = loss.backward()
        nn.backward(dLdZ)
        nn.step_Adam()
    mean_loss = np.mean(losses)
    print(f'Iterasjon {j} L = {mean_loss}')

Iterasjon 0 L = 0.27666469158167284
Iterasjon 1 L = 0.27761103543094245
Iterasjon 2 L = 0.27589695001252723
Iterasjon 3 L = 0.2763400229802328
Iterasjon 4 L = 0.2756879304060388
Iterasjon 5 L = 0.27691849613581987
Iterasjon 6 L = 0.27666469377477165
Iterasjon 7 L = 0.27697432174159164
Iterasjon 8 L = 0.27545465189005613
Iterasjon 9 L = 0.27677295613687
Iterasjon 10 L = 0.27472345257378195
Iterasjon 11 L = 0.2753485051192104
Iterasjon 12 L = 0.2743096119763916
Iterasjon 13 L = 0.27495454825971605
Iterasjon 14 L = 0.2746010186221402
Iterasjon 15 L = 0.27556448079206114
Iterasjon 16 L = 0.27821785944984867
Iterasjon 17 L = 0.2768633661398169
Iterasjon 18 L = 0.27499330299029523
Iterasjon 19 L = 0.2758060240576397
Iterasjon 20 L = 0.27356927829884853
Iterasjon 21 L = 0.27242641294350994
Iterasjon 22 L = 0.2715903271452821
Iterasjon 23 L = 0.2717643549330026
Iterasjon 24 L = 0.2711656762665881
Iterasjon 25 L = 0.2711585973436827
Iterasjon 26 L = 0.2706154123504656
Iterasjon 27 L = 0.2709591

#### Tester for en itterasjon for å se om den greier å predikere neste verdi

In [4]:
print(np.argmax(nn.forward(onehot(np.array([[1, 0 ,0, 0, 1]]), m)),axis=1))

[[0 0 0 0 0]]


#### Tester hvor mange riktige prediksjoner vi får

In [5]:
x_test = data['x_test']
y_test = data['y_test']


def predict(nn: NeuralNetwork, xs: dict, r:int, m:int):
    
    batches = xs.shape[0]
    y = np.zeros((xs.shape[0],xs.shape[1],r))
    for i in range(batches):
        x = xs[i]
        for j in range(r):
            #print(j)
            X = onehot(x,m)
            z = nn.forward(X)
            Z = np.argmax(z, axis=1)
            #print(Z.shape)
            #print(x.shape)
            toAppend =Z[:,-1:]
            x = np.append(x, toAppend, axis=1)
        y[i,:,:] = x[:,:-r]
    return y



y_pred = predict(nn, x_test, r, m)

def count_correct_predictions(y_pred, y_test):
    correct_predictions = np.count_nonzero(np.count_nonzero(y_pred == y_test, axis=2) == y_pred.shape[-1])
    return correct_predictions

y_test = data['y_test']
correct_predictions = count_correct_predictions(y_pred, y_test)
print("Antall rette prediksjoner:", correct_predictions)
print("Totalt antall prediksjoner:", y_pred.shape[1])


Antall rette prediksjoner: 169
Totalt antall prediksjoner: 250


Når to forskjellige verdier kan forekomme på fem ulike steder, resulterer dette i totalt $32$ mulige kombinasjoner ($2^5 = 32$). Denne mangfoldigheten gjør det praktisk talt umulig å teste algoritmen vår med nye sekvenser. Ideelt sett ville tapet tendert mot null, og prediksjonene ville vært korrekte hver gang, siden algoritmen burde gjenkjenne det riktige svaret ($y$) i stedet for å forutsi neste sekvens. Dette antyder at vektene våre kanskje ikke er optimalt tilpasset.